In [ ]:
from os.path import expanduser, join
import numpy as np
import matplotlib.pyplot as plt

from black_box_tools.db_utils import DBUtils
from black_box_tools.data_utils import DataUtils
from black_box_tools.plot_utils import PlotUtils

# Parameters

The following parameters should be set appropriately so that the correct data are read.

In [ ]:
log_db_data_name = 'data'

# Database restoring

Uncomment the lines below for restoring multiple MongoDB databases.

In [ ]:
# data_dirs = ['/path/to/db/dump/1', '/path/to/db/dump/2']
# subdb_names = ['db1', 'db2']
# DBUtils.restore_subdbs(db_name=log_db_data_name,
#                        data_dirs=data_dirs,
#                        subdb_names=subdb_names)

We now read the metadata of the sub-databases so that we can generate appropriate multi-database plots later.

In [ ]:
subdb_metadata = DBUtils.get_subdb_metadata(log_db_data_name)

# Data parsing

We first read the data of interest - in this case the velocity commands.

In [ ]:
cmd_vel_docs = DBUtils.get_subdb_docs(log_db_data_name, 'ros_ropod_cmd_vel', subdb_metadata)

timestamps = {}
x_velocities = {}
y_velocities = {}
theta_velocities = {}
for subdb in cmd_vel_docs:
    timestamps[subdb] = DataUtils.get_all_measurements(cmd_vel_docs[subdb], 'timestamp')
    x_velocities[subdb] = DataUtils.get_all_measurements(cmd_vel_docs[subdb], 'linear/x')
    y_velocities[subdb] = DataUtils.get_all_measurements(cmd_vel_docs[subdb], 'linear/y')
    theta_velocities[subdb] = DataUtils.get_all_measurements(cmd_vel_docs[subdb], 'angular/z')

# Data plotting

In [ ]:
fig = plt.figure(figsize=(20,5))

last_index = 0
for subdb in cmd_vel_docs:
    subdb_indices = np.linspace(last_index, last_index+len(timestamps[subdb]), len(timestamps[subdb]))
    PlotUtils.subplot_data(fig=fig, subplot_params=(1,3,1),
                           timestamps=subdb_indices,
                           data=x_velocities[subdb],
                           x_label='t',
                           y_label='Velocity ($m/s$)',
                           data_label=subdb)

    PlotUtils.subplot_data(fig=fig, subplot_params=(1,3,2),
                           timestamps=subdb_indices,
                           data=y_velocities[subdb],
                           x_label='t',
                           y_label='Velocity ($m/s$)',
                           data_label=subdb)

    PlotUtils.subplot_data(fig=fig, subplot_params=(1,3,3),
                           timestamps=subdb_indices,
                           data=theta_velocities[subdb],
                           x_label='t',
                           y_label='Velocity ($rad/s$)',
                           data_label=subdb)

    last_index += len(timestamps[subdb])

plt.show()